За да стартирате следващите тетрадки, ако все още не сте го направили, трябва да разположите модел, който използва `text-embedding-ada-002` като базов модел и да зададете името на разполагането във файла .env като `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

След това ще заредим Индекса на вграждане в Pandas Dataframe. Индексът на вграждане е съхранен в JSON файл, наречен `embedding_index_3m.json`. Индексът на вграждане съдържа вгражданията за всеки от транскриптите на YouTube до края на октомври 2023 г.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

След това ще създадем функция, наречена `get_videos`, която ще търси в Индекса на вградените представяния за заявката. Функцията ще връща топ 5 видеа, които са най-подобни на заявката. Функцията работи по следния начин:

1. Първо се създава копие на Индекса на вградените представяния.
2. След това се изчислява вграденото представяне за заявката, използвайки OpenAI Embedding API.
3. След това в Индекса на вградените представяния се създава нова колона, наречена `similarity`. Колоната `similarity` съдържа косинусното сходство между вграденото представяне на заявката и вграденото представяне за всеки видео сегмент.
4. След това Индексът на вградените представяния се филтрира по колоната `similarity`. Индексът се филтрира, за да включва само видеа, които имат косинусно сходство по-голямо или равно на 0.75.
5. Накрая Индексът на вградените представяния се сортира по колоната `similarity` и се връщат топ 5 видеа.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Тази функция е много проста, тя просто отпечатва резултатите от заявката за търсене.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Първо, Индексът на вграждане се зарежда в Pandas Dataframe.
2. След това на потребителя се предлага да въведе заявка.
3. След това се извиква функцията `get_videos`, за да се търси в Индекса на вграждане за заявката.
4. Накрая се извиква функцията `display_results`, за да се покажат резултатите на потребителя.
5. След това на потребителя се предлага да въведе друга заявка. Този процес продължава, докато потребителят не въведе `exit`.

![](../../../../translated_images/bg/notebook-search.1e320b9c7fcbb0bc.png)

Ще бъдете подканени да въведете заявка. Въведете заявка и натиснете enter. Приложението ще върне списък с видеоклипове, които са релевантни на заявката. Приложението също така ще върне връзка към мястото във видеото, където се намира отговорът на въпроса.

Ето някои заявки, които можете да изпробвате:

- Какво е Azure Machine Learning?
- Как работят свързващите невронни мрежи?
- Какво е невронна мрежа?
- Мога ли да използвам Jupyter Notebooks с Azure Machine Learning?
- Какво е ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:  
Този документ е преведен с помощта на AI преводаческа услуга [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи могат да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или неправилни тълкувания, произтичащи от използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
